## Tensorflow的Layer基础知识介绍

Layer是构成Model的基本单元，简单理解成一个数据处理函数：
* Layer可以输入一个数据tensor，经过代码处理，输出一个数据Tensor
* Layer中有参数Weights，Model训练的目标，就是提供数据(features、label），优化Model中的Layer的Weights

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

### 1. Layer就像函数一样调用

In [2]:
# Dense即全连接层
layer = layers.Dense(3, activation='relu')

In [3]:
# 输入数据为一个tensor
inputs = tf.random.uniform(shape=(5, 6))
inputs

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0.40407777, 0.88063526, 0.25358224, 0.97589326, 0.06460178,
        0.0016253 ],
       [0.80155647, 0.6136501 , 0.9499092 , 0.84700835, 0.09532583,
        0.00904214],
       [0.32065988, 0.76755106, 0.6785172 , 0.7970929 , 0.24687922,
        0.7238227 ],
       [0.6344321 , 0.66100574, 0.344859  , 0.37777865, 0.33043253,
        0.622627  ],
       [0.24514043, 0.7800337 , 0.9093611 , 0.83008635, 0.44269657,
        0.04932714]], dtype=float32)>

In [4]:
# 执行函数变换，注意输入输出的形状也变化了
layer(inputs)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.9739331 , 0.69509286, 0.        ],
       [0.11392275, 0.87058806, 0.        ],
       [1.0093035 , 1.0945739 , 0.        ],
       [0.7901717 , 0.7247521 , 0.        ],
       [0.6692092 , 1.3004112 , 0.        ]], dtype=float32)>

### 2. Layer本身常用的方法

In [5]:
# 虽然Layer可以像函数一样调用处理输入得到输出，但是它本身维护了可以更新的Weights参数
layer.weights

[<tf.Variable 'dense/kernel:0' shape=(6, 3) dtype=float32, numpy=
 array([[-0.3564902 , -0.33182186,  0.04517931],
        [ 0.81089985,  0.80152786, -0.6774622 ],
        [-0.64842594,  0.71471524, -0.23070705],
        [ 0.5463972 , -0.0853045 , -0.17682546],
        [ 0.5299891 ,  0.38956988, -0.7267979 ],
        [ 0.5178119 ,  0.10034859,  0.08581448]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [6]:
# trainable_weights会在训练期间，通过梯度下降进行更新
layer.trainable_weights

[<tf.Variable 'dense/kernel:0' shape=(6, 3) dtype=float32, numpy=
 array([[-0.3564902 , -0.33182186,  0.04517931],
        [ 0.81089985,  0.80152786, -0.6774622 ],
        [-0.64842594,  0.71471524, -0.23070705],
        [ 0.5463972 , -0.0853045 , -0.17682546],
        [ 0.5299891 ,  0.38956988, -0.7267979 ],
        [ 0.5178119 ,  0.10034859,  0.08581448]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [7]:
# 以含有Numpy矩阵的列表形式返回层的权重
layer.get_weights()

[array([[-0.3564902 , -0.33182186,  0.04517931],
        [ 0.81089985,  0.80152786, -0.6774622 ],
        [-0.64842594,  0.71471524, -0.23070705],
        [ 0.5463972 , -0.0853045 , -0.17682546],
        [ 0.5299891 ,  0.38956988, -0.7267979 ],
        [ 0.5178119 ,  0.10034859,  0.08581448]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [8]:
layer.kernel

<tf.Variable 'dense/kernel:0' shape=(6, 3) dtype=float32, numpy=
array([[-0.3564902 , -0.33182186,  0.04517931],
       [ 0.81089985,  0.80152786, -0.6774622 ],
       [-0.64842594,  0.71471524, -0.23070705],
       [ 0.5463972 , -0.0853045 , -0.17682546],
       [ 0.5299891 ,  0.38956988, -0.7267979 ],
       [ 0.5178119 ,  0.10034859,  0.08581448]], dtype=float32)>

In [9]:
layer.bias

<tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

In [10]:
# 包含层配置的字典
layer.get_config()

{'name': 'dense',
 'trainable': True,
 'dtype': 'float32',
 'units': 3,
 'activation': 'relu',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

### 3. 自己实现Dense全连接层

Dense 实现以下操作： output = activation(dot(input, kernel) + bias)：
* kernel 是由网络层创建的权值矩阵
* bias 是其创建的偏置向量
* activation 是按逐个元素计算的激活函数

In [11]:
class SimpleDense(tf.keras.layers.Layer):
    """要实现一个自己的Layer，首先继承tf.keras.Layer"""
    
    def __init__(self, units=32, activation=tf.keras.activations.relu):
        """在这里实现跟输入数据无关的初始化，units是输出单元个数"""
        super(SimpleDense, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        """这里你已经知道了输入shape，做一些其它初始化"""
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'), trainable=True)
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(self.units,), dtype='float32'), trainable=True)

    def call(self, inputs):
        """在这里执行真正的计算，m*n @ n*k = m*k"""
        return self.activation(tf.matmul(inputs, self.w) + self.b)

In [12]:
simpleDense = SimpleDense(3)

In [13]:
# 输入数据
inputs = tf.random.uniform(shape=(5, 6))
inputs

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0.68861127, 0.41894233, 0.37655258, 0.5538323 , 0.6926129 ,
        0.5943234 ],
       [0.11853802, 0.7858939 , 0.41474915, 0.04745328, 0.7790879 ,
        0.5131738 ],
       [0.33588636, 0.6533185 , 0.9034835 , 0.6472776 , 0.8411925 ,
        0.3909695 ],
       [0.14168155, 0.6888174 , 0.10918486, 0.31171548, 0.0862596 ,
        0.66624594],
       [0.7140136 , 0.763389  , 0.03812623, 0.35847807, 0.03051162,
        0.48225987]], dtype=float32)>

In [14]:
simpleDense(inputs)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.06213272, 0.00960668, 0.02161434],
       [0.05979951, 0.        , 0.03843728],
       [0.04832684, 0.05169922, 0.08261046],
       [0.07751171, 0.        , 0.00511048],
       [0.05392166, 0.        , 0.        ]], dtype=float32)>